# Library Visualizations
##### Data visualizations of print books in the University of Edinburgh's library

* **Funding:** Edinburgh Futures Institute (EFI)
* **Data Source:** Library and University Collections (L&UC), OCLC MARC Data for Print Books
* **Project Dates:** 17 June 2019 - 27 July 2019
* **Author:** Lucy Havens

#### This notebook creates a dataframe of library metadata (in MARC format) and then exports a .csv file of the metadata

Load relevant libraries

In [43]:
import csv
import pandas as pd
import re

Load .mrc data as a .json files (exported using MarcEdit's OpenRefine Export feature)

In [44]:
# from: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_json.html
#   when typ == 'frame', allowed orients are {'split','records','index', 'columns','values', 'table'}
#   when typ == 'series', allowed orients are {'split','records','index'}

marc_series0 = pd.read_json('Data/ZIP_FILE2/BatchOf5/JSON/msplit00000000.json', typ='series', orient='index')
marc_series1 = pd.read_json('Data/ZIP_FILE2/BatchOf5/JSON/msplit00000001.json', typ='series', orient='index')
marc_series2 = pd.read_json('Data/ZIP_FILE2/BatchOf5/JSON/msplit00000002.json', typ='series', orient='index')
marc_series3 = pd.read_json('Data/ZIP_FILE2/BatchOf5/JSON/msplit00000003.json', typ='series', orient='index')
marc_series4 = pd.read_json('Data/ZIP_FILE2/BatchOf5/JSON/msplit00000004.json', typ='series', orient='index')

series_list = [marc_series0, marc_series1, marc_series2, marc_series3, marc_series4]

# for elem in marc_series:
#     for subelem in elem:
#         print(subelem)
#         print()


Each row in the .csv file should correspond with a single "RecordNumber" value.

Column names in the .csv file should correspond with "Tag" + subfield ("$x" in "Content") combinations.

In [45]:
# FOR ALL MARC DATA:
#   Extract column names from a list of pandas series created from JSON files of MARC data 
#   Create list of indeces based on the number of unique RecordNumbers (book) in each JSON file


def columnsFromJSON(json_series_list):
    csv_cols = []
    csv_indeces_lists = []
    i = 0
    for json_series in json_series_list:
        csv_indeces = []
        for elem in json_series:
            for subelem in elem:
                record_no = subelem['RecordNumber']
                if (record_no not in csv_indeces) and (record_no != ''):
                    csv_indeces.append(record_no)
                # Construct column names
                tag = subelem['Tag']
                # In MARC, subfields are indicated with a dollar sign and lower case letter
                subfields_list = re.findall("\$\w{1}",subelem['Content'])
                for subfield in subfields_list:
                    marc_field = tag+subfield
                    if marc_field not in csv_cols:
                        csv_cols += [marc_field]
            
        csv_indeces_lists += [csv_indeces]
    
    return [csv_cols,csv_indeces_lists]


[column_names,row_indeces] = columnsFromJSON(series_list)
total_rows = len(row_indeces[0]) + len(row_indeces[1]) + len(row_indeces[2]) + len(row_indeces[3]) + len(row_indeces[4])

print("Total Columns:", len(column_names))
print("Total Records:", total_rows)

Total Columns: 582
Total Records: 5000


In [46]:
# FOR SUBSET OF MARC DATA:
#   Metadata to export in .csv file for visualization:

marc_tags_fields = ['020$a', '020$c', '041$a', '043$a', '045', '050$a', '072$a', '082$a',
       '100$a', '100$e', '130$a', '130$l', '240$a', '240$f', '240$l', '245$a',
       '245$b', '245$c', '260$a', '260$b', '260$c', '264$a', '264$b', '264$c',
       '300$a', '336$a', '336$b', '600$a', '600$c', '600$d', '600$t', '600$x',
       '600$z', '630$a', '630$l', '630$v', '630$x', '650$a', '650$x', '650$y',
       '650$z', '651$a', '651$x', '651$y', '651$z', '655$a', '655$a.1',
       '655$y']

marc_names = ['isbn', 'isbn1', 'text_language', 'geographic_area_code',
       'time_period_of_content', 'loc_call_no', 'subject_cat_code', 'dewey_no',
       'personal_name', 'relator', 'uniform_title', 'real_world_obj_URI',
       'uniform_title1', 'date_of_work', 'real_world_obj_URI1', 'title',
       'title_cont', 'stat_of_resp', 'publ_place', 'publisher', 'publ_date',
       'publ_place1', 'publisher1', 'publ_date1', 'physical_desc',
       'content_type_term', 'content_type_code', 'personal_name1',
       'titles_words_assoc_w_name', 'dates_assoc_w_name', 'title1',
       'general_subdivision', 'geo_subdivision', 'uniform_title1',
       'real_world_obj_URI2', 'form_subdivision', 'general_subdivision1',
       'topic_or_geo', 'general_subdivision2', 'chrono_subdivision',
       'geo_subdivision1', 'geo_name', 'general_subdivision3',
       'chrono_subdivision1', 'geo_subdivision2', 'genre_form_focus',
       'genre_form_focus1', 'chrono_subdivision2']

field_name_dict = dict(zip(marc_tags_fields,marc_names))
# field_name_dict

In [47]:
# INPUT: pandas series read from a JSON file, a MARC field, and a list of record numbers
# OUTPUT: list of column values (one per record number) for the inputted MARC field (col_name)
def colValuesFromJSON(json_series, col_name, csv_index):
    col_values = {}
    col_values = col_values.fromkeys(csv_index,"Unknown")  # 1000
    for elem in json_series:
        for subelem in elem:
            record_no = subelem['RecordNumber']
            
            # Ignore empty items in json_series
            if (record_no != ''):

                tag = subelem['Tag']

                # Determine whether the json_series subelement has relevant content for the inputted column name
                if tag in col_name:
                    
                    # If the column name is a MARC subfield, extract relevant subcontent...
                    content = subelem['Content']
                    if ('$' in col_name) and ('$' in content):
                        # Obtain the subfield
                        subfield = col_name[-1]
                        # Find the content data associated with the subfield
                        subcontent = re.findall("\$"+subfield+"{1}[^$]+",content)  # returns a list
                        subfield = '$' + subfield
                        for val in subcontent:
                            col_values[record_no] = val.replace(subfield,'')
                    # ...otherwise, add all content
                    else:
                        col_values[record_no] = subelem['Content']

    return list(col_values.values())

Extract MARC data from the json series as is relevant to the subset of data of interest ("marc_tags_fields"), adding the data to a DataFrame one column (MARC field) at a time.

In [78]:
df = pd.DataFrame()
for tag_field in marc_tags_fields:
    i = 0
    maxI = len(series_list)
    col_vals = []
    while i < maxI:
        col_vals += colValuesFromJSON(series_list[i],tag_field,row_indeces[i])
        i+= 1
    
    col_name = field_name_dict[tag_field]  # obtain the MARC field name from the MARC field code
    df[col_name] = col_vals
df

,isbn,isbn1,text_language,geographic_area_code,time_period_of_content,loc_call_no,subject_cat_code,dewey_no,personal_name,relator,...,general_subdivision2,chrono_subdivision,geo_subdivision1,geo_name,general_subdivision3,chrono_subdivision1,geo_subdivision2,genre_form_focus,genre_form_focus1,chrono_subdivision2
0,Unknown,Unknown,grclat,Unknown,Unknown,PA3872,Unknown,Unknown,"Apollonius,",Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
1,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,"Lamond, Peter,",Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
2,9781403976277 (hbk.),Unknown,Unknown,n-us---,Unknown,HB119.F84,Unknown,Unknown,"Ebenstein, Alan O.",Unknown,...,Unknown,Unknown,United States,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
3,9780230007895,Unknown,Unknown,e-it---,Unknown,HB109.M63,Unknown,B,"Szenberg, Michael.",Unknown,...,Unknown,Unknown,Italy.,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
4,1403996237,Unknown,Unknown,e-uk---,Unknown,HB103.K47,Unknown,330.156,"Davidson, Paul,",Unknown,...,Unknown,Unknown,Great Britain.,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
5,1842143050,Unknown,Unknown,Unknown,Unknown,RG136,Unknown,613.94,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
6,Unknown,Unknown,lat,Unknown,Unknown,PA3443,Unknown,Unknown,"Schneidewin, Friedrich Wilhelm,",Unknown,...,Unknown,Unknown,Greece.,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
7,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,"King, Nicholas,",Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
8,Unknown,Unknown,Unknown,Unknown,Unknown,PA3872.Z4,Unknown,Unknown,"Rzach, Aloisius.",Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
9,9781842773215 (pbk.),Unknown,eng,n-us---,Unknown,HB501,Unknown,Unknown,"Amin, Samir.",Unknown,...,Political aspects.,Unknown,Unknown,United States,Foreign relations,1989-,Unknown,Unknown,Unknown,Unknown


In [79]:
df.shape

(5000, 47)

Export a CSV file of the DataFrame

[2]